In [1]:
import os
import numpy as np
import xarray as xray
import netCDF4
from scipy import stats
from scipy import io
from scipy import fftpack as fft
from scipy import linalg as lin
from scipy import signal as sig
from netCDF4 import Dataset
from scipy import interpolate as naiso

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
basedir = '/data/scratch/takaya/POP_data/'

In [5]:
fname_Mar = os.path.join(basedir, 
                         'Surface_Daily/hybrid_v5_rel04_BC5_ne120_t12_pop62.pop.h.nday1.0046-03-01.nc')
ex = xray.open_dataset(fname_Mar, decode_times=False)
maskT = ex.KMT > 1
maskU = ex.KMU > 1
latT = ex.TLAT.where(maskT).values
lonT = ex.TLONG.where(maskT).values
latU = ex.ULAT.where(maskU).values
lonU = ex.ULONG.where(maskU).values
dxT = 1e-2 * ex.DXT.where(maskT).values
dyT = 1e-2 * ex.DYT.where(maskT).values
dxU = 1e-2 * ex.DXU.where(maskU).values
dyU = 1e-2 * ex.DYU.where(maskU).values

In [6]:
dz = np.diff(ex.z_t)[0]*1e-2

In [7]:
tarea = ex.TAREA.values
tarea_r = 1e4 * np.ma.masked_invalid(tarea**-1).filled(0.)

dxt_r = 1e2 * ex.DXT.values**-1
dyt_r = 1e2 * ex.DYT.values**-1
dxu_r = 1e2 * ex.DXU.values**-1
dyu_r = 1e2 * ex.DYU.values**-1

        
p5 = .5
c2 = 2.
hus = 1e-2 * ex.HUS.values
hte = 1e-2 * ex.HTE.values
huw = 1e-2 * ex.HUW.values
htn = 1e-2 * ex.HTN.values

uarea = ex.UAREA.values
uarea_r = 1e4 * np.ma.masked_invalid(uarea**-1).filled(0.)


###########
# coefficients for \nabla**2(U) (without metric terms)
###########
work1 = hus * hte**-1
dus = work1 * uarea_r
dun = np.roll(work1, 1, axis=0) * uarea_r
work1 = huw * htn**-1
duw = work1 * uarea_r
due = np.roll(work1, 1, axis=1) * uarea_r

###########
# coefficients for metric terms in \nabla**2(U, V)
###########
kxu = (np.roll(huw, 1, axis=1) - huw) * uarea_r
kyu = (np.roll(hus, 1, axis=0) - hus) * uarea_r

#East-West
work1 = (hte - np.roll(hte, -1, axis=1)) * tarea_r
work2 = np.roll(work1, 1, axis=1) - work1
dxkx = p5 * (work2 + np.roll(work2, 1, axis=0)) * dxu_r
work2 = np.roll(work1, 1, axis=0) - work1
dykx = p5 * (work2 + np.roll(work2, 1, axis=1)) * dyu_r

# North-South
work1 = (htn - np.roll(htn, -1, axis=0)) * tarea_r
work2 = np.roll(work1, 1, axis=0) - work1
dyky = p5 * (work2 + np.roll(work2, 1, axis=1)) * dyu_r
work2 = np.roll(work1, 1, axis=1) - work1
dxky = p5 * (work2 + np.roll(work2, 1, axis=0)) * dxu_r

            
dum = - (dxkx + dyky + c2*(kxu**2 + kyu**2))
dmc = dxky - dykx


###########      
# coefficients for metric mixing terms which mix U,V.
###########
dme = c2*kyu * (htn + np.roll(htn, 1, axis=1))**-1
dmn = -c2*kxu * (hte + np.roll(hte, 1, axis=0))**-1

duc = - (dun + dus + due + duw)
dmw = -dme
dms = -dmn

In [8]:
j_eq = np.argmin(ex.ULAT[:,0].values**2)
amf = (uarea / ex.UAREA.values[j_eq, 0])**1.5     #uarea is in [cm^2]
amf = np.ma.masked_array(amf, ~maskU.values).filled(0.) 

In [9]:
def vector_laplacian(U, V, mask, dun, dus, due, duw, dmc, dmn, dms, dme, dmw):
    """ \nabla**2(U or V) at momentum points
    """
    cc = duc + dum
    
    lap_u = (cc * U +
            dun * np.roll(U, -1, axis=1) + 
            dus * np.roll(U, 1, axis=1) + 
            due * np.roll(U, -1, axis=2) + 
            duw * np.roll(U, 1, axis=2) +
            dmc * V + 
            dmn * np.roll(V, -1, axis=1) + 
            dms * np.roll(V, 1, axis=1) + 
            dme * np.roll(V, -1, axis=2) + 
            dmw * np.roll(V, 1, axis=2)
            )
    lap_v = (cc * V +
            dun * np.roll(V, -1, axis=1) + 
            dus * np.roll(V, 1, axis=1) + 
            due * np.roll(V, -1, axis=2) + 
            duw * np.roll(V, 1, axis=2) +
            dmc * U + 
            dmn * np.roll(U, -1, axis=1) + 
            dms * np.roll(U, 1, axis=1) + 
            dme * np.roll(U, -1, axis=2) + 
            dmw * np.roll(U, 1, axis=2)
            )
    if lap_u.ndim == 3:
        for t in range(lap_u.shape[0]):
            lap_u[t] = np.ma.masked_array(lap_u[t], mask).filled(0.)
            lap_v[t] = np.ma.masked_array(lap_v[t], mask).filled(0.)
    else:
        lap_u = np.ma.masked_invalid(lap_u[t], mask).filled(0.)
        lap_v = np.ma.masked_invalid(lap_v[t], mask).filled(0.)
            
    return lap_u, lap_v


def biharmonic_tendency(U, V, mask, dun, dus, due, duw, 
                        dmc, dmn, dms, dme, dmw, amf, am=-2.7e10):
    """Calculate tendency due to biharmonic diffusion.
    """
    d2uk, d2vk = [amf * t for t in vector_laplacian(U, V, mask, 
                                                    dun, dus, due, duw, dmc, dmn, dms, dme, dmw)]
    hduk, hdvk = [am * t for t in vector_laplacian(d2uk, d2vk, mask, 
                                                    dun, dus, due, duw, dmc, dmn, dms, dme, dmw)]
    if hduk.ndim == 3:
        for t in range(hduk.shape[0]):
            hduk[t] = np.ma.masked_array(hduk[t], mask).filled(0.)
            hdvk[t] = np.ma.masked_array(hdvk[t], mask).filled(0.)
    else:
        hduk = np.ma.masked_array(hduk, mask).filled(0.)
        hdvk = np.ma.masked_array(hdvk, mask).filled(0.)
    return hduk, hdvk

# Winter

In [21]:
#########
month = 1
#########
##########
# daystart = 1
# daylag = 13
# Nmonth = 31
##########
# year = range(34)
# years = np.append(year, range(35,41))
years = range(1)
fname_mat = ['' for x in range(len(years))]

t = 0
for yearnum in years:
    fname = os.path.join(basedir,
                                 'Surface_Daily/hybrid_v5_rel04_BC5_ne120_t12_pop62.pop.h.nday1.%04d-%02d-01.nc' 
                                 % (yearnum+46, month))
    fname_mat[t] = fname
    t += 1
    
print fname_mat

['/data/scratch/takaya/POP_data/Surface_Daily/hybrid_v5_rel04_BC5_ne120_t12_pop62.pop.h.nday1.0046-01-01.nc']


In [22]:
num = 0

for fname in fname_mat:
    
    data = xray.open_dataset(fname, decode_times=False)
    U = 1e-2 * data.U1_1.where(maskU)
    V = 1e-2 * data.V1_1.where(maskU)
    hduk, hdvk = biharmonic_tendency(U.values, V.values, ~maskU, dun, dus, due, duw, 
                        dmc, dmn, dms, dme, dmw, amf)
    hduk = xray.DataArray( hduk, dims=U.dims, coords=U.coords )
    hdvk = xray.DataArray( hdvk, dims=V.dims, coords=V.coords )
    savename = os.path.join(basedir, 'Momentum_Biharmonic_closure%04d_%02d' % (years[num]+46, month))
    np.savez(savename, hduk=hduk, hdvk=hdvk)
    
    num += 1

In [ ]:
#########
month = 2
#########
##########
# daystart = 1
# daylag = 13
# Nmonth = 31
##########
year = range(34)
years = np.append(year, range(35,41))
fname_mat = ['' for x in range(len(years))]

t = 0
for yearnum in years:
    fname = os.path.join(basedir,
                                 'Surface_Daily/hybrid_v5_rel04_BC5_ne120_t12_pop62.pop.h.nday1.%04d-%02d-01.nc' 
                                 % (yearnum+46, month))
    fname_mat[t] = fname
    t += 1
    
print fname_mat

In [ ]:
num = 0

for fname in fname_mat:
    
    data = xray.open_dataset(fname, decode_times=False)
    U = 1e-2 * data.U1_1.where(maskU)
    V = 1e-2 * data.V1_1.where(maskU)
    hduk, hdvk = biharmonic_tendency(U.values, V.values, ~maskU, dun, dus, due, duw, 
                        dmc, dmn, dms, dme, dmw, amf)
    hduk = xray.DataArray( hduk, dims=U.dims, coords=U.coords )
    hdvk = xray.DataArray( hdvk, dims=V.dims, coords=V.coords )
    savename = os.path.join(basedir, 'Momentum_Biharmonic_closure%04d_%02d' % (years[num]+46, month))
    np.savez(savename, hduk=hduk, hdvk=hdvk)
    
    num += 1

In [ ]:
#########
month = 3
#########
##########
# daystart = 1
# daylag = 13
# Nmonth = 31
##########
year = range(34)
years = np.append(year, range(35,41))
fname_mat = ['' for x in range(len(years))]

t = 0
for yearnum in years:
    fname = os.path.join(basedir,
                                 'Surface_Daily/hybrid_v5_rel04_BC5_ne120_t12_pop62.pop.h.nday1.%04d-%02d-01.nc' 
                                 % (yearnum+46, month))
    fname_mat[t] = fname
    t += 1
    
print fname_mat

In [ ]:
num = 0

for fname in fname_mat:
    
    data = xray.open_dataset(fname, decode_times=False)
    U = 1e-2 * data.U1_1.where(maskU)
    V = 1e-2 * data.V1_1.where(maskU)
    hduk, hdvk = biharmonic_tendency(U.values, V.values, ~maskU, dun, dus, due, duw, 
                        dmc, dmn, dms, dme, dmw, amf)
    hduk = xray.DataArray( hduk, dims=U.dims, coords=U.coords )
    hdvk = xray.DataArray( hdvk, dims=V.dims, coords=V.coords )
    savename = os.path.join(basedir, 'Momentum_Biharmonic_closure%04d_%02d' % (years[num]+46, month))
    np.savez(savename, hduk=hduk, hdvk=hdvk)
    
    num += 1

# Summer

In [ ]:
for month in range(7,10):

    fname_mat = ['' for x in range(len(years))]

    t = 0
    for yearnum in years:
        fname = os.path.join(basedir,
                                     'Surface_Daily/hybrid_v5_rel04_BC5_ne120_t12_pop62.pop.h.nday1.%04d-%02d-01.nc' 
                                     % (yearnum+46, month))
        fname_mat[t] = fname
        t += 1

    num = 0
    for fname in fname_mat:

        data = xray.open_dataset(fname, decode_times=False)
        U = 1e-2 * data.U1_1.where(maskU)
        V = 1e-2 * data.V1_1.where(maskU)
        hduk, hdvk = biharmonic_tendency(U.values, V.values, ~maskU, dun, dus, due, duw, 
                            dmc, dmn, dms, dme, dmw, amf)
        hduk = xray.DataArray( hduk, dims=U.dims, coords=U.coords )
        hdvk = xray.DataArray( hdvk, dims=V.dims, coords=V.coords )
        savename = os.path.join(basedir, 'Momentum_Biharmonic_closure%04d_%02d' % (years[num]+46, month))
        np.savez(savename, hduk=hduk, hdvk=hdvk)

        num += 1